In [130]:
import pandas as pd

In [131]:
sp = pd.read_csv('../Database/ETC/sp500.csv', index_col=0)

In [132]:
index = sp['Adj Close'].pct_change().dropna()

In [133]:

col = ['06/1999-12/1999', '01/2000-12/2002(9/11)', '01/2003-12/2006', '01/2007-12/2009(GFC)',
       '01/2010-12/2019', '01/2020-12/2022(Covid-19)', 'Overall']

period = [range(1, 100), range(100, 285), range(285, 333), range(333, 369), range(369, 380), range(380, 391),
          range(1, 391)]

In [134]:
# index.T.to_csv('../Database/sp500_return.csv')

In [135]:
import os
import numpy as np
import pandas as pd
from PCA_and_ETC import read_and_preprocess_data, generate_PCA_Data

cl_list = ['60_characteristics_us', '70_characteristics_us', '80_characteristics_us', '90_characteristics_us']
# cl_list = [f'{i}_characteristics_us' for i in range(10, 80, 10)]
for a in cl_list:
    input_dir = f'../Database/Clustering_Result/Contrastive_Learning/{a}'
    input_dir2 = f'../Database/Clustering_Result/Contrastive_Learning/Prob/{a}'
    output_dir = f'../Database/Clustering_Result/Contrastive_Learning/CL_Pre/{a}'

    files = sorted(filename for filename in os.listdir(input_dir))

    for file in files:
        data = read_and_preprocess_data(input_dir, file)
        prob = read_and_preprocess_data(input_dir2, file)
        prob.index = prob['firms']
        prob = prob.drop(columns='firms')
        prob_filtered = prob[(prob.max(axis=1) <= prob.max(axis=1).quantile(0.80))]
        data.loc[prob_filtered.index, 'clusters'] = 0
        data.to_csv(os.path.join(output_dir, file))

In [164]:
from PCA_and_ETC import read_and_preprocess_data, generate_PCA_Data
import numpy as np
import pandas as pd

input_dir = '../Database/Clustering_Result/DBSCAN'
input_dir2 = '../Database/characteristics_US'
subdirectories = [d for d in os.listdir(input_dir)]

for subdir in subdirectories:
    base_directory = f'{input_dir}/{subdir}'
    files = sorted(filename for filename in os.listdir(base_directory) if filename.endswith('.csv'))

    bic_dict = {}

    for file in files:
        df = read_and_preprocess_data(base_directory, file)
        df2 = read_and_preprocess_data(input_dir2, file)
        df2 = generate_PCA_Data(df2)

        n = df.shape[0]
        d = df2.shape[1]

        clusters = {}
        total_length = 0
        for j in range(1 + max(set(df['clusters']))):
            # if j == 0:
            #     continue
            cluster_data = df2.loc[df[df['clusters'] == j].index]

            # 각 클러스터에 대하여 정규화된 특성들의 var의 평균을 구함
            clusters[j] = cluster_data.var().mean()

        # total variance는 클러스터 전체의 var의 평균을 사용.
        total_variance = pd.DataFrame(clusters.values()).mean()

        # 최대 가능도
        L = np.exp(-0.5 * total_variance)
        # 모델의 자유도
        k = len(clusters)-1
        v = d - np.log(n/k)
        # BIC 계산
        bic = -2 * np.log(L) + v * np.log(n)
        bic_dict[file] = bic

    print(subdir)
    # 전체기간에 대하여 bic score의 평균을 구함
    a = pd.DataFrame(bic_dict.values()).mean()
    print(a)

0.1
0    646.001832
dtype: float64



KeyboardInterrupt

